<a href="https://colab.research.google.com/github/pramodith/llm_exploration/blob/pramodith%2Friddllama/riddle_reasoning_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Riddle LLama
This notebook trains a reasoning model to answer riddles. Riddles are reasoning heavy driven tasks. A model needs to be able to learn to associate different facts/concepts together to coherently come up with the right answer.

In [1]:
%pip install unsloth==2025.5.7 vllm flashinfer-python datasets litellm scikit-learn bitsandbytes ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.3 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 99.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 5.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 2.2 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━

In [2]:
import unsloth
from datasets import load_dataset, Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-23 12:24:20.247473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748003060.447210      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748003060.505373      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-23 12:24:37 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-23 12:24:37 [__init__.py:239] Automatically detected platform cuda.


In [20]:
# riddles_dataset = load_dataset("mlfoundations-dev/riddle_sense")
riddles_dataset  = load_dataset("Hypersniper/riddles_v1")["train"]

README.md:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

riddles.json:   0%|          | 0.00/555k [00:00<?, ?B/s]

riddles_2.json:   0%|          | 0.00/112k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/469 [00:00<?, ? examples/s]

In [21]:
riddles_dataset[7]

{'answer': 'cards',
 'output': "The answer to this question lies in the realm of playing cards. Let's break down the logic. \n\nThe number 13 is significant in a deck of cards as each suit has 13 cards: Ace through 10, and the three picture cards: Jack, Queen, and King. Now, let's think about the 'hearts' part of the question. In a standard deck of cards, there are four suits: hearts, diamonds, clubs, and spades. One of these suits is hearts.\n\nTherefore, in a deck of cards, there are 13 hearts. However, these hearts do not beat as they are not living, they are simply a suit in a deck of cards. They are symbolic hearts, not biological ones. So, based on this logical deduction, we can conclude that a deck of cards is what has 13 hearts but none that beat. \n\nThis type of riddle requires both literal and metaphorical thinking. The number 13 and the word 'hearts' might initially lead one to think of something biological or living because hearts are typically associated with living being

In [3]:
# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format, the answer section must be as concise as possible and all the thinking/reasoning should be within the think tags:
<think>
...
</think>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<think>
{think}
</think>
<answer>
{answer}
</answer>
"""


In [4]:
reformatted_riddles_dataset = riddles_dataset.map(
    lambda x: {
        "question": x["instruction"],
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": x["instruction"]},
        ],
    },
)

NameError: name 'riddles_dataset' is not defined

Format the dataset to contain a __prompt__ key and an __answer__ key.

In [4]:
def ml_foundations_reformat():
    reformatted_riddles_dataset = []
    labels = {}
    for doc in riddles_dataset["train"]:
        for choice in doc["question"]["choices"]:
            labels[choice["label"]] = choice["text"]

        answer = labels[doc["answerKey"]]
        prompt= [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": doc["question"]["stem"]},
        ]
        reformatted_riddles_dataset.append({
            "question": doc["question"]["stem"],
            "prompt": prompt,
            "answer": answer,
        })

In [6]:
from dotenv import load_dotenv
load_dotenv()

False

In [5]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["HF_TOKEN"] = ""

## Dataset Filtering

In [6]:
import litellm
from litellm import batch_completion
litellm.num_retries = 2  # Retry 5 times

from jinja2 import Environment

environment = Environment()
riddle_quality_system_prompt = """
You are a helpful assistant that evaluates the quality of a riddle.
You will be given a question and its answer. Score the question and answer based on the
following rubric:

0 if the question is not a riddle or the answer is not a riddle answer.
1 if the riddle is very easy and the answer is obvious.
2 if the riddle is moderately hard to solve.
3 if the riddle is hard and needs a lot of think and multiple associations to get to the answer.

Here are some examples:
Question: Where does a person put their phone when they are walking?
Answer: In their pocket.
Score: 0

Question: What has keys but can't open locks?
Answer: A piano.
Score: 1

Question: What has a heart that doesn't beat?
Answer: An artichoke.
Score: 2

Question: What has a head and a tail but no body?
Answer: A coin.
Score: 1

Question: Many minds, but not a single face,
I make decisions with layered grace.
Each vote I cast makes outcomes clear,
With randomness keeping bias in fear.
What am I?
Answer: Random forest.
Score: 3
"""

In [ ]:
reformatted_riddles_dataset[0:2]

In [7]:
import time
from tqdm import tqdm
def score_riddles(dataset):
    scores = []
    for doc in tqdm(range(0, len(dataset), 16)):

        questions, answers =  zip(*[(q, a) for q, a in zip(dataset[doc:doc+16]["question"], dataset[doc:doc+16]["answer"])])
        responses = batch_completion(
            model = "openai/gpt-4.1-2025-04-14",
            temperature=0.0,
            max_tokens=6,
            messages = [
                [
                    {"role": "system", "content": riddle_quality_system_prompt},
                    {"role": "user", "content": f"Please score the following riddle:\nQuestion :{question}\nAnswer: {answer}\nScore: " }
                ]
                for question, answer in zip(questions, answers)
            ],
        )
        time.sleep(10)
        try:
            scores.extend([r.choices[0].message.content for r in responses])
        except Exception as e:
            print(e)
            return responses
    return scores

In [8]:
import re
def extract_score(scores, dataset):
    extracted_scores = []
    for ind, score in enumerate(scores):
        if isinstance(score, str):
            score = re.findall(r"Score: (\d)", score)
            # print(score)
            if len(score) == 0:
                extracted_scores.append(-1)
            else:
                extracted_scores.append(int(score[0]))

    return extracted_scores


In [ ]:
scores = score_riddles(reformatted_riddles_dataset)
# reformatted_riddles_dataset = extract_score(scores, reformatted_riddles_dataset)

In [ ]:
scores

In [ ]:
extracted_scores = extract_score(scores, reformatted_riddles_dataset)
reformatted_riddles_dataset = reformatted_riddles_dataset.add_column("quality_score", extracted_scores)

In [ ]:
reformatted_riddles_dataset[0]

In [ ]:
reformatted_riddles_dataset = Dataset.from_list(reformatted_riddles_dataset)
reformatted_riddles_dataset.save_to_disk("./riddles_dataset")
reformatted_riddles_dataset.push_to_hub("Pramodith/riddles_dataset_scored", private=True, token=True)

Load scored/graded dataset

In [8]:
reformatted_riddles_dataset = load_dataset("Pramodith/riddles_dataset_scored", token=True)["train"]
reformatted_riddles_dataset = reformatted_riddles_dataset.remove_columns(["output"])

README.md:   0%|          | 0.00/529 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/417k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/469 [00:00<?, ? examples/s]

In [9]:
for row in reformatted_riddles_dataset:
    if len(row["prompt"])>500:
        print(row["prompt"])
        print("\n\n")

In [10]:
reformatted_riddles_dataset = [dict(row) for row in reformatted_riddles_dataset if row["quality_score"] > 1]

In [11]:
reformatted_riddles_dataset[0]

{'answer': 'music',
 'instruction': 'At the sound of me, one may dream or stamp their feet, At the sound of me, one may laugh or sometimes weep.',
 'question': 'At the sound of me, one may dream or stamp their feet, At the sound of me, one may laugh or sometimes weep.',
 'prompt': [{'content': '\nRespond in the following format, the answer section must be as concise as possible and all the thinking/reasoning should be within the think tags:\n<think>\n...\n</think>\n<answer>\n...\n</answer>\n',
   'role': 'system'},
  {'content': 'At the sound of me, one may dream or stamp their feet, At the sound of me, one may laugh or sometimes weep.',
   'role': 'user'}],
 'quality_score': 2}

Create train, val and test splits of the dataset.

In [12]:
from sklearn.model_selection import train_test_split
train, not_train = train_test_split(reformatted_riddles_dataset, test_size=0.2, random_state=42)
dev, test = train_test_split(not_train, test_size=0.5, random_state=42)

In [13]:
# Convert list of dicts to a huggingface dataset
from datasets import Dataset
train_dataset = Dataset.from_list(train)
dev_dataset = Dataset.from_list(dev)
test_dataset = Dataset.from_list(test)

In [14]:
train_dataset[0]

{'answer': 'nose',
 'instruction': 'Two little holes in the side of a hill. Just as you come to the cherry-red mill.',
 'question': 'Two little holes in the side of a hill. Just as you come to the cherry-red mill.',
 'prompt': [{'content': '\nRespond in the following format, the answer section must be as concise as possible and all the thinking/reasoning should be within the think tags:\n<think>\n...\n</think>\n<answer>\n...\n</answer>\n',
   'role': 'system'},
  {'content': 'Two little holes in the side of a hill. Just as you come to the cherry-red mill.',
   'role': 'user'}],
 'quality_score': 2}

In [15]:
print(f"Train size: {len(train)}")
print(f"Dev size: {len(dev)}")
print(f"Test size: {len(test)}")

Train size: 305
Dev size: 38
Test size: 39


In [16]:
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_xml_think(text: str) -> str:
    think = text.split("<think>")[-1]
    think = think.split("</think>")[0]
    return think.strip()


### Unsloth
Load up `Qwen 2.5 3B Instruct`, and set parameters

In [17]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer think traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    layers_pattern=r"model.layers\.\d+\.(q_proj|k_proj|v_proj|o_proj|gate_proj|up_proj|down_proj)$",  # This is a pattern to match layer modules

    layers_to_transform = list(range(28, 36)),
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.5.7: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 79.24%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 224.
Unsloth: vLLM's KV Cache can use up to 9.26 GB. Also swap space = 4 GB.
WARNING 05-23 12:24:48 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 05-23 12:25:02 [config.py:717] This model supports multiple tasks: {'score', 'reward', 'classify', 'e

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.5.7 patched 36 layers with 36 QKV layers, 8 O layers and 8 MLP layers.


### Reward functions

In [18]:
import random
# Reward functions
def correctness_reward_func(prompts, completions, answer, quality_score, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    if random.random() < 0.05:
        print('-'*20)
        print(f"Question:\n{q}")
        print(f"Correct Answer:\n{answer[0]}")
        print(f"Responses:\n{responses[0]}")
        print(f"Extracted:\n{extracted_responses[0]}")
    
    return [1 + qs if r.strip() != "" and (a.lower() in r.lower() or r.lower() in a.lower()) else 0 for r, a, qs in zip(extracted_responses, answer, quality_score)]

def answer_length_reward_func(prompts, completions, answer, quality_score, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    rewards  = []
    for r, a in zip(extracted_responses, answer):
        rewards.append(0)
        if a in r or r in a:
            if len(r) > len(a):
                rewards[-1] = -0.001 * (len(r) - len(a))
        else:
            if len(r) < len(a):
                rewards[-1] = -0.005 * (len(a) - len(r))
    return rewards


def think_length_reward_func(prompts, completions, answer, quality_score, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_think(r) for r in responses]
    rewards = [0.5 if 100 < len(r) < 1000 else -0.5 for r in extracted_responses]
    return rewards

def count_xml(text) -> float:
    count = 0.0
    if text.count("<think>") == 1:
        count += 0.125
    if text.count("</think>") == 1:
        count += 0.125
    if text.count("<answer>") == 1:
        count += 0.125
    if text.count("</answer>") == 1:
        count += 0.125
    return count

def xmlcount_reward_func(prompts, completions, answer, quality_score, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

In [ ]:
%pip show unsloth

## Model Training

In [27]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 1e-5,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    do_eval = False,
    # eval_strategy='steps',
    # per_device_eval_batch_size=8,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 512,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 500,
    # eval_steps = 5,
    save_steps = 10,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        correctness_reward_func,
        answer_length_reward_func,
        think_length_reward_func
    ],
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 305 | Num Epochs = 4 | Total steps = 500
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 26,607,616/3,000,000,000 (0.89% trained)


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / correctness_reward_func,rewards / answer_length_reward_func,rewards / think_length_reward_func
1,0.000000,0.450625,0.480890,156.250000,0.000016,0.390625,0.000000,-0.002500,0.062500
2,0.000000,0.578125,0.276045,141.875000,0.000010,0.453125,0.000000,0.000000,0.125000
3,0.000000,1.205438,0.742555,211.250000,0.000009,0.414062,0.937500,-0.083625,-0.062500
4,0.000000,0.688812,0.833957,160.500000,0.000009,0.390625,0.187500,-0.014313,0.125000
5,0.000000,0.268438,0.449814,191.687500,0.000009,0.460938,0.000000,-0.005000,-0.187500
6,0.000000,0.787500,0.791586,150.000000,0.000016,0.414062,0.187500,-0.001562,0.187500
7,0.000000,0.358750,0.502627,168.187500,0.000009,0.421875,0.000000,-0.000625,-0.062500
8,0.000000,0.522812,0.455082,165.625000,0.000010,0.398438,0.000000,-0.000625,0.125000
9,0.000000,0.056562,0.513021,154.000000,0.000009,0.390625,0.000000,-0.459062,0.125000
10,0.000000,0.843188,1.164039,246.062500,0.000011,0.406250,0.562500,-0.000562,-0.125000


--------------------
Question:
If you drop me I'm sure to crack. But give me a smile and I'll always smile back.
Correct Answer:
mirror
Responses:
<think>
The phrase "If you drop me I'm sure to crack" suggests that the object is fragile when it falls. "But give me a smile and I'll always smile back" implies that upon receiving a smile, the object remains in the same state. Given these clues, the object is most likely a mirror.
</think>
<answer>
Mirror
Extracted:
Mirror


Unsloth: Input IDs of length 517 > the model's max sequence length of 512.
We shall truncate it ourselves. It's imperative if you correct this issue first.


--------------------
Question:
I can be written, I can be spoken, I can be exposed, I can be broken.
Correct Answer:
news
Responses:
<think>
The given sentence fits the properties of a word: it can be written (using a keyboard), spoken (using the mouth), exposed (published), and broken (partially rearranged). Words commonly exhibit these characteristics.
</think>
<answer>
A word.
Extracted:
A word.


In [ ]:
test_dataset[0]

In [51]:
from torch.utils.data import DataLoader
import torch

def evaluate_on_test_set(model, tokenizer, test_dataset):
    model.eval()
    
    all_rewards = []
    total_samples = 0
    
    with torch.no_grad():
        for row in tqdm(test_dataset):
            # Generate responses
            # Note: This depends on your exact dataset format
            prompt = row['prompt']  # Adjust based on your dataset structure
            # Generate completions
            inputs = tokenizer.apply_chat_template(prompt, return_tensors="pt", tokenize=True, padding=True, truncation=True, add_generation_prompt=True)
            outputs = model.generate(
                inputs.to("cuda:0"),
                max_new_tokens=512,
                do_sample=True,
                temperature=0.7,
                pad_token_id=tokenizer.eos_token_id
            )
            
            # Decode responses
            response = tokenizer.decode(outputs[0][len(inputs[0]): ], skip_special_tokens=True)
            
            # Calculate rewards
            accuracy_rewards = 0
            format_rewards = 0
            
            accuracy_rewards += correctness_reward_func([prompt], [[{"content":response}]], [row["answer"]], [row["quality_score"]])[0]
            format_rewards += xmlcount_reward_func([prompt], [[{"content":response}]], [row["answer"]], [row["quality_score"]])[0]
    
    
    return {
        'accuracy_rewards': accuracy_rewards/len(test_dataset),
        'format_rewards':  format_rewards/len(test_dataset)
    }

# Usage
test_results = evaluate_on_test_set(
    model=trainer.model,
    tokenizer=tokenizer,
    test_dataset=test_dataset
)

print(f"Test Results: {test_results}")

 13%|█▎        | 5/39 [00:37<04:20,  7.65s/it]

--------------------
Question:
Two brothers we are, great burdens we bear. All day we are bitterly pressed. Yet this I will say, we are full all the day, and empty when go to rest.
Correct Answer:
boots
Responses:
<think>
The brothers described in the riddle are a mill and wheat. The mill grinds the wheat, which takes all day and leaves no residue at the end, symbolizing "full all the day, and empty when go to rest." The mill, as a machine, works hard all day but ends up empty after finishing its task.
</think>
<answer>
The answer to the riddle is a mill and wheat. The mill grinds the wheat all day, consuming its input but leaving no remainder, thus "bitterly pressed" and "full all the day." However, by the end of the day, the mill is "empty" as it has no more wheat to grind. At night, it is "rested" but "empty" as it has nothing to do.
Extracted:
The answer to the riddle is a mill and wheat. The mill grinds the wheat all day, consuming its input but leaving no remainder, thus "bitterl

 23%|██▎       | 9/39 [01:05<03:59,  7.98s/it]

--------------------
Question:
I cut through evil like a double edged sword, and chaos flees at my approach. Balance I single-handedly upraise, through battles fought with heart and mind, instead of with my gaze.
Correct Answer:
justice
Responses:
<think>
The phrase suggests a powerful and influential figure, likely a hero or a deity, who wields control over chaos and evil. The description also implies a focus on mental and moral strength rather than physical prowess. The lines evoke imagery of a protector who stands for justice and order. 

The expression "cut through evil like a double-edged sword" indicates someone powerful and capable, whereas "chaos flees at my approach" suggests a commanding presence. "Balance I single-handedly upraise" further emphasizes their role in maintaining order. 

The last line, "through battles fought with heart and mind, instead of with my gaze," reinforces the idea of a leader who uses both internal fortitude and strategic planning to achieve their go

100%|██████████| 39/39 [04:59<00:00,  7.69s/it]

Test Results: {'accuracy_rewards': 0.0, 'format_rewards': 0.009615384615384616}


In [ ]:
model.save_lora("grpo_saved_lora")

In [ ]:
model.save_pretrained_merged("riddle_qwen2.5-1.5B", tokenizer, save_method = "merged_16bit")

In [ ]:
import os
hf_token = os.getenv("hf_token")

In [ ]:
model.push_to_hub_merged("Pramodith/riddle_qwen2.5-3B", tokenizer, save_method = "merged_16bit", token = hf_token)


In [ ]:
sample_riddle = "Question:\nI have keys but open no locks,I have space but no room, You can enter, but I’m not a door. What am I?"

messages = [{"role": "system", "content":SYSTEM_PROMPT}, {"role": "user", "content":sample_riddle}]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
)
output_tokens = model.generate(torch.LongTensor([text]).to("cuda:0"), max_new_tokens=256, do_sample=False)

In [ ]:
model

In [ ]:
!pip install lighteval[math]
# !pip install flashinfer-python -i https://flashinfer.ai/whl/cu126/torch2.6/ --no-deps

In [ ]:
!lighteval vllm "pretrained=Pramodith/riddle_qwen2.5-3B,dtype=float16,max_model_length=32768,gpu_memory_utilization=0.8,generation_parameters={max_new_tokens:32768,temperature:0.6,top_p:0.95}" "lighteval|aime24|0|0" \
    --use-chat-template \
    --output-dir .